Just some initial research

In [1]:
#from sklearn.datasets import make_hastie_10_2

#X, y = make_hastie_10_2(random_state=0)
#print(type(X))
#print(type(y))
#X_train, X_test = X[:2000], X[2000:]
#y_train, y_test = y[:2000], y[2000:]

#clf = HistGradientBoostingRegressor(max_iter=100).fit(X_train, y_train)
#clf.score(X_test, y_test)
#X_to_predict = [[1.76405235, 0.40015721, 0.97873798, 2.2408932, 1.86755799, -0.97727788, 0.95008842, -0.15135721, -0.10321885, 0.4105985]]
#print(X_to_predict)
#clf.predict(X_to_predict)

# Importing necessary libraries and load the data

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# configs
pd.set_option('display.max_columns', None) # we want to display all columns in this notebook
pd.set_option('display.max_rows', 100) # increase rows to be displayed
pd.set_option('display.max_colwidth', None) # show full cell contents

df_train = pd.read_csv('../input/child-mind-institute-problematic-internet-use/train.csv')
df_test = pd.read_csv('../input/child-mind-institute-problematic-internet-use/test.csv')
df_sub = pd.read_csv('../input/child-mind-institute-problematic-internet-use/sample_submission.csv')
df_dict = pd.read_csv('../input/child-mind-institute-problematic-internet-use/data_dictionary.csv')

# Save all categorical features into a list
These features are saved to a list. This list will be used later.

In [3]:
categorical_features = []

for i in range(1, 81):
    sample_type = df_dict.iloc[i]['Type']
    if 'str' in sample_type:
        #categorical_features.append(i-1)
        categorical_features.append(df_dict.iloc[i]['Field'])

# include PCIAT_Season feature or not
categorical_features.remove('PCIAT-Season')
#print(categorical_features)

# Save all PCIAT features into a list
These features are saved to a list. This list will be used later.

In [4]:
pciat_features = []

for i in range(1, 81):
    sample_type = df_dict.iloc[i]['Field']
    if 'PCIAT' in sample_type:
        #categorical_features.append(i-1)
        pciat_features.append(df_dict.iloc[i]['Field'])

# include PCIAT_Season feature or not
#pciat_features.remove('PCIAT-Season')

# this make model overfit
# pciat_features.remove('PCIAT-PCIAT_Total')

print(pciat_features)

['PCIAT-Season', 'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20', 'PCIAT-PCIAT_Total']


# Clean up the train set
The train set needs some modifications before being fed to the model. First, drop all data samples with no sii as these samples has no contribution to the prediction. Next, drop the irrelevant column - id. Finally, drop all PCIAT features and categorical features. We tried using the one-hot edcoding method to handle the categorical features but it did not improve much. Therefore, these features are dropped. In method 1, we tried predicting sii without PCIAT features, so we also dropped these features.

In [5]:
df_train = df_train.dropna(subset=['sii']).drop(columns=['id'] + pciat_features + categorical_features)

# Start training the model
Here, we used the HistGradientBoostingRegressor model for the problem and MSE to evaluate it.

In [6]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score


def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

#df_train = pd.get_dummies(df_train, columns=categorical_features)

y = df_train.sii
X = df_train.drop(columns = ['sii'])

#print(X.info())

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0)
model = HistGradientBoostingRegressor(max_iter=100).fit(X_train, y_train)

#model.score(X_val, y_val)
pred = model.predict(X_val)
quadratic_weighted_kappa(y_val, pred)

0.5264200503605809

# Clean up the test set
Just like the train set, we drop irrelevant column 'id' and all the categorical features. The PCIAT features aren't in the test set.

In [7]:
cleaned_df_test = df_test.drop(columns=['id'] + categorical_features)

# Predict the test set
The data in the test set is predicted here.

In [8]:
#df_test = pd.get_dummies(df_test, columns=categorical_features)

#print(df_test.info())

X_test = cleaned_df_test
y_test = df_sub.sii

pred = model.predict(X_test)
pred = pred.round().astype(int)
print(pred)
print(y_test)
#mean_absolute_error(pred, y_test)

[1 0 0 1 1 1 0 0 1 0 1 0 1 0 1 1 0 0 1 1]
0     0
1     1
2     2
3     3
4     0
5     1
6     2
7     3
8     0
9     1
10    2
11    3
12    0
13    1
14    2
15    3
16    0
17    1
18    2
19    3
Name: sii, dtype: int64


# Prepare file for submission
Export the predicted data to a file, ready for submission.

In [9]:
# Save test predictions to file
output = pd.DataFrame({'id': df_test.id.values,
                       'sii': pred.round().astype(int)})
output.to_csv('submission.csv', index=False)